<a href="https://colab.research.google.com/github/alexanderjoe/CS1164-FinalProject/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 1164 Machine Learning - Final Project

Group Members:
- Alexander Diaz
- Gavin Wieckowski

(0) Import required dependencies for the project

In [5]:
# Setup Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# PCA Dependencies
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

(1) Load And Clean Data

In [10]:
# todo

(2) Dimensionality Reduction

In [11]:
#todo

(3) Training Model

In [12]:
# todo

(4) Evaluate Model

In [ ]:
# todo

(5) Lessons Learned

TODO...